# 处理信用卡账单数据  
- 问题：  
  1. 时间错乱
  2. 数字太乱  
  3. 可能出现账单的断裂
- 目标：  
  1. 归序时间  
  2. 优化数字的显示方式  
  3. 拼接账单链
- 步骤：
  1. 排序-为每个用户每张信用卡的账单  
  2. 完善账单链，并补充时间，间隔先设置为30天
  3. 更改金额数字

In [ ]:
import pandas as pd
import numpy as np
import copy

# 变量定义
N = 20000           # 定义新生成样本的数量
LimitBanks = 4      # 每个人最多有4个银行卡
indBenQi = 6        # 本期账单余额所在列
indShangQi = 3      # 上期账单金额所在列
indShangQiHuan = 4  # 上期还款金额所在列
indEdu = 5          # 信用卡额度所在列
indTime = 2         # 时间戳所在的列
pAddrec = 0.8       # 加入新纪录作为粘合两条断裂链的概率
newidStart = 69495  # 新样本的编号，从这个数字开始

## 数据预处理
- 有重复值。$\color{red}{删除}$重复值
- 不同银行简单统计。
  1. 因为数据来自不同银行，会发现一些银行的数据比较诡异，故需查看。  
    bankid = 1，3100条记录，缺失值非常多    
    bankid = 2，13.5万条记录，相对质量较高  
    bankid = 3，22.1万条记录，基本没有上期账单金额和还款金额  
    bankid = 4，26.5万条记录  
    bankid = 510014和510044，有循环利息，但数据量较少
  2. $\color{red}{删除}$部分银行的记录。  
    编号为1、12，以及所有大于50000的银行id，数量都较少。
- 需直接$\color{red}{删除}$的字段
  1. 消费笔数：经常出现消费笔数为0，但账单余额不为0的情况。  
  2. 可用余额：出现数量非常少，在某个银行id下会较多。但此数字非常奇怪。  
  3. 调整金额，循环利息，预借现金额度：基本上都是空值，仅在少量小众银行下有值。   
  4. 本期账单金额：数值非常奇怪，感觉跟其余的字段完全没有关系。  
  5. 本期账单最低还款额：基本没有什么用处。  
- 需判断后处理的记录
  1. 信用卡额度：有些用户会有较大波动，估计是数据合并时出现错误。需检查波动情况，然后删除此用户此银行的所有记录。

In [ ]:
'''
  第1步，读入数据，删除无用字段，并删除重复值
'''
datapath = '../data/'
train_credit = pd.read_csv(datapath + 'train/bill_detail_train.txt', header = None)
test_credit = pd.read_csv(datapath + 'test/bill_detail_test.txt', header = None)
train_credit = pd.concat([train_credit,test_credit],axis = 0)
train_credit.columns = ['用户id','账单时间戳','银行id','上期账单金额','上期还款金额','信用卡额度','本期账单余额',  
                     '本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额',  
                     '预借现金额度','还款状态']
print('原始数据总规模为：', train_credit.shape)
remvVars = ['消费笔数','可用余额','调整金额','循环利息','预借现金额度','本期账单金额', '本期账单最低还款额']
train_credit.drop(remvVars, axis=1, inplace=True)
train_credit.drop_duplicates(inplace=True)
print('删除无用字段和重复值之后的数据总规模为：', train_credit.shape)
train_credit.drop
train_credit.head()

In [ ]:
'''
  第2步，删除指定银行的数据
'''
remvBankid = [1,3,12,510014,510016,510017,510022,510024,510025,510026,510027,510033,510037,510044,510050,510053,510057]
train_credit = train_credit.set_index(['银行id'])
train_credit.drop(remvBankid,inplace=True)
train_credit.reset_index()

In [ ]:
'''
  第3步，删除信用卡额度变化太快的字段
  
  方法：取出每一对（用户id，银行id）对应的记录，然后判断其众数，若众数数量不超过
  min(6,总记录数量的一半)，则不保留此数据
'''
# 新建一个df，存放被保留的数据
temp_df = pd.DataFrame()
# 取出所有用户id
user_lst = list(set(train_credit['用户id'].tolist()))
i = 1
for usr in user_lst:
    i += 1
    if i % 1000 == 0:
        print('已完成处理%d个用户...'%i)
    # 取出用户usr的所有记录
    usr_df = train_credit[train_credit['用户id']==usr]
    # 取出用户usr的所有银行id
    bank_lst = list(set(usr_df['银行id'].tolist()))
    for bank in bank_lst:
        # 取出用户usr&银行bank的所有记录
        usr_bank_df = usr_df[usr_df['银行id']==bank]
        if usr_bank_df.shape[0] < 3:
            temp_df = pd.concat([temp_df,usr_bank_df], axis=0)
        else:
            # 求得“信用卡额度”的众数出现的次数
            limit_count = usr_df['信用卡额度'].value_counts()
            limit_mostFreqCount = list(limit_count)[0]
            if limit_mostFreqCount < min(6,int(usr_df['信用卡额度'].shape[0]/2)):
                # 跳过这个usr_bank_df
                pass
            else:
                temp_df = pd.concat([temp_df,usr_bank_df], axis=0)
        
print('删除异常信用卡额度之后的数据总规模为：', temp_df.shape)        

In [ ]:
'''
  最后，把处理好的数据写入文件，备用
''' 
temp_df.to_csv('Billdata_clean_20190817.csv')

## 排序-单个用户单个银行
- 方法：
  1. 以原数据的时间作为基本排序
  2. 根据“上期账单账单金额”与“本期账单金额”的对应关系，优化时间序列
  3. 若存在断链的情况，则找出所有可以连起来的链；再做进一步的判断
- 处理流程：  
  请见同文件下文件名：农业银行数据-信用卡账单处理流程图

In [ ]:
# 1. 读入新数据
bill_df = pd.read_csv('Billdata_clean_20190817.csv')
bill_df.head()

In [ ]:
# 2 定义排序函数
def sortBillList(df_usr_bank):
    # 由于dataframe类型，不便于重复删除指定行，且控制好指针，故转为二维数组
    arr = df_usr_bank.sort_values(by='账单时间戳').values
        
    # bill链可能会出现断裂，此时会找到所有连续的链，存放于resL。
    resL = []             # resL定义为一个三维数组
    while True:
        if len(arr) == 0:
            return resL
        elif len(arr) == 1:
            L = [list(arr[0])]
            resL += [L,]
            return resL
        else:
            L = []
            L = L + [list(arr[0]),]   #在L中加入arr中的第一条 然后进行判断
            arr = np.delete(arr,0,axis=0)
            i = 0
            
            while i < len(arr):
                if arr[i][indShangQi] == L[len(L)-1][indBenQi]:   
                    L = L + [list(arr[i]),]
                    arr = np.delete(arr,i,axis=0)
                    i = 0
                elif arr[i][indBenQi] == L[0][indShangQi]:   
                    L = [list(arr[i]),] + L
                    arr = np.delete(arr,i,axis=0)
                    i = 0
                else:
                    i += 1
            # 然后判断df是否为空
            resL = resL + [L,]   



In [ ]:
# 提出所有用户id； 建议分几批来运行，速度会快很多。每次生成的数据，保存成npy即可。
userSet = list(set(bill_df['用户id'].tolist()))

allSortedList = []

# 遍历每一位用户
i = 1
for usr in userSet:
    if i % 1000 == 0:
        print('已处理%d个用户...'%i)
    i += 1
    df_usr = bill_df.loc[bill_df['用户id'] == usr]
    # 遍历此用户的每一个银行id
    bankSet = set(df_usr['银行id'].tolist())
    for bank in bankSet:
        df_usr_bank = df_usr.loc[df_usr['银行id'] == bank]
        # 调用排序算法,返回生成该“用户&银行”的所有list，存入allSortedList
        allSortedList = allSortedList + [sortBillList(df_usr_bank)]
    

# 将数据保存
np.save('tempBillChain.npy',allSortedList)

## 合并BillChains
- 背景  
  对于当前的数据，对用户u银行i的所有信用卡账单，断裂非常严重，需要补成链状。  
- 合并的顺序  
  以信用卡额度为标准，小额度的尽可能放前面。
- 方法  
  1. 按较高的概率，添加一条记录，把两条断裂的链粘合起来
  2. 按较小的概率，直接把两条链放在一起，就当作是记录缺失了  
  
  具体的方案，可以看当前目录下的流程图，合并Billchains.jpg
  

In [ ]:
# 读入排好顺序的BillChain
allSortedList = np.load('tempBillChain.npy')

In [ ]:
# 获得每一个连续链的第一个非0信用卡额度值
def getValidCreditLimit(lst2d, ind):
    res = 0
    for templst in lst2d:
        if templst[ind] > res:
            res = templst[ind]
            break
    return res

# 开始合并
mergedChains = []
iout = 1
for usr_bank_list in allSortedList:
    if iout % 1000 == 0:
        print('已处理%d个用户...'%iout)
    iout += 1
    
    resL = usr_bank_list[0]
    # 获取当前resL中首个大于0的额度
    begCreditLimit = getValidCreditLimit(resL, indEdu)
            
    for i in range(1,len(usr_bank_list)):
        # 获取当前usr_bank_list[i]记录中，第一个不为0的额度
        thisCreditLimit = getValidCreditLimit(usr_bank_list[i], indEdu)
        
        if thisCreditLimit < begCreditLimit:           
            # 把第i条记录加在resL“头部”
            if random.random() < pAddrec:
                # 新增记录，链接resL[0]和usr_bank_list[i]
                newrecord = usr_bank_list[i][len(usr_bank_list[i])-1][:]
                # 新增记录的上期金额 = usr_bank_list[i][end][本期]
                newrecord[indShangQi] = usr_bank_list[i][len(usr_bank_list[i])-1][indBenQi]   
                # 新增记录的本期余额 = resL[0][上期]
                newrecord[indBenQi] = resL[0][indShangQi]                 
                # 新增记录的上期还款金额，在上期金额上，进行浮动，[-0.2，0.2]
                newrecord[indShangQiHuan] = newrecord[indShangQi] + (random.random()-0.5)/2
                resL = [newrecord] + resL
            resL = usr_bank_list[i] + resL
        else:
            # 把第i条记录加在resL“尾部”
            if random.random() < pAddrec:
                # 新增记录，链接resL[end]和usr_bank_list[i]
                newrecord = resL[len(resL)-1][:]
                # 新增记录的上期金额 = resL[end][本期]
                newrecord[indShangQi] = resL[len(resL)-1][indBenQi] 
                # 新增记录的本期余额 = usr_bank_list[i][0][上期]
                newrecord[indBenQi] = usr_bank_list[i][0][indShangQi]  
                # 新增记录的上期还款金额，在上期金额上，进行浮动，[-0.2，0.2]
                newrecord[indShangQiHuan] = newrecord[indShangQi] + (random.random()-0.5)/2
                resL = resL + [newrecord]
            resL = resL + usr_bank_list[i]
            begCreditLimit = thisCreditLimit
    
    mergedChains = mergedChains + [resL,]

# 把得到的结果保存下来，原始数据的记录
np.save('initialUserBillChainP'+str(pAddrec)+'.npy',mergedChains)

## 生成新样本
- 目的与原则
  1. 补充新的样本数据
  2. 需要区分正负样本，12000个负样本，8000个正样本
- 方法
  1. 为每一个用户，安排1~4个银行
  2. 对于新的正样本用户，仅选择正样本用户的信用卡账单情况；负样本用户同样仅对应负样本
  3. 在原有记录上，仅删除样本数据  
  
  具体的删除样本方案，可以看当前目录下的流程图，生成新样本.jpg

In [ ]:
# 先读入数据
initialBillChains = np.load('initialUserBillChainP'+str(pAddrec)+'.npy')
# 根据标签数据 'overdue_train.txt'，将initialBillChains分成两部分
labels = pd.read_csv('../data/train/overdue_train.txt',header=None)
labels.columns = ['用户id','label']

iniBCpos, iniBCneg = [],[]
ii = 1
for lst in initialBillChains:
    if ii % 1000 == 0:
        print('已完成%d条，共%d条...'%(ii,len(initialBillChains)))
    ii += 1
    usrid = lst[0][1]
    try:
        if labels[labels['用户id']==usrid]['label'].tolist()[0] == 1:
            iniBCpos = iniBCpos + [lst]
        else:
            iniBCneg = iniBCneg + [lst]
    except:
        pass

In [ ]:
def GenerateSamples(iniBC,idStart,Nsample):
    newUserBC = []
    for i in range(Nsample):
        if (i+1)%500 == 0:
            print('已新建%d个用户...'%i)
        r = np.random.randint(LimitBanks) + 1
        newid = idStart + i
        banks = []
        for _ in range(r):
            # 随机取出一条记录，应是一个二维数组
            tempind =  np.random.randint(len(iniBC))
            thisChain = copy.deepcopy( iniBC[tempind] )
            # 取出这条记录的bankid
            bankid = thisChain[0][0]
            if bankid in banks:
                continue
            # 开始改造这条记录
            banks.append(bankid)

            ####### 改造1：删除记录
            if np.random.random() < 0.5:
                # 只删除头或尾
                x = min(7,int((len(thisChain)-1)/3) )
                xhead = int(x/2)
                xtail = x-xhead
                thisChain = np.delete(thisChain,range(xhead),axis=0)
                thisChain = np.delete(thisChain,range(len(thisChain)-xtail,len(thisChain)),axis=0)
            else:
                # 混合删除
                a = min(7,int((len(thisChain)-1)/3) )
                y = min(3,a)
                if a>3:
                    x = a-3
                    xhead = int(x/2)
                    xtail = x-xhead
                    thisChain = np.delete(thisChain,range(xhead),axis=0)
                    thisChain = np.delete(thisChain,range(len(thisChain)-xtail,len(thisChain)),axis=0)
                # 删除y个中间值
                remvInds = np.random.randint(0,len(thisChain),y)
                thisChain = np.delete(thisChain,remvInds,axis=0)

            ####### 改造2：替换userid
            thisChain[:,1] = newid

            ####### 改造3：上期金额、本期余额、信用卡额度，统一添加一个随机数
            ################### 但原来为0的，不能加
            temprand = (np.random.random() - 0.5)/2
            thisChain[:,indBenQi] = thisChain[:,indBenQi] + temprand
            thisChain[:,indShangQi] = thisChain[:,indShangQi] + temprand
            thisChain[:,indEdu] = thisChain[:,indEdu] + temprand

            ####### 改造4：上期还款金额，各自改一个小小的随机数
            thisChain[:,indShangQiHuan] = thisChain[:,indShangQiHuan] + temprand/10
            
            thisChain[(thisChain < 0.5) & (thisChain>-0.5)] = 0
            
            newUserBC = newUserBC + [thisChain,]
    return newUserBC

In [ ]:
newBCneg = GenerateSamples(iniBCneg,newidStart,12000)
newBCpos = GenerateSamples(iniBCpos,newidStart+12000,8000)

In [ ]:
np.save('newUserBCneg.npy',newBCneg)
np.save('newUserBCpos.npy',newBCpos)

## 样本进一步加工
- 方法
  1. 以较大概率，0.6，将一些负值置为0  
  2. 若时间戳为0，则以0.5的概率将此条记录删除  
  3. 若时间戳不为0，则在原来时间上做一些修改，找到第一个不为0的时间，往后看每一条记录  
     3.1 若与前一条连续，则加(29.8~31.2)天的偏移。  
     3.2 如不连续，则加(29.8~31.2)天的偏移。  
     注：当前时间戳，单位为秒，因此需要 $随机数*24*3600$

In [ ]:
def Process(billChains):
    thisBC = []
    for lst in billChains:
        # 遍历此记录
        lst = np.array(lst)
        i = 0
        while i < len(lst):
            if lst[i][indTime] == 0:
                if np.random.random() < 0.5:
                    lst = np.delete(lst, i, axis=0)
                    i -= 1
                elif np.random.random() < 0.6:
                    # 将负值置为0
                    lst[lst < 0] = 0
            else:
                if np.random.random() < 0.6:
                    # 将负值置为0
                    lst[lst < 0] = 0
                try:
                    if lst[i][indBenQi] == lst[i+1][indShangQi]:
                        timelag = int((np.random.random()*(31.2-29.8) + 29.8) * 24 * 3600)
                        lst[i+1][indTime] = lst[i][indTime] + timelag
                    else:
                        timelag = int((np.random.random()*(31.2-29.8) + 29.8) * 24 * 3600 * np.random.randint(2,4))
                        lst[i+1][indTime] = lst[i][indTime] + timelag
                except:
                    pass
            i += 1
        thisBC = thisBC + [lst]
    return thisBC
            

In [ ]:
# 处理newUserBCpos
newBCpos = np.load('newUserBCpos.npy')
newBCpos1 = Process(newBCpos)
# 处理newUserBCneg
newBCneg = np.load('newUserBCneg.npy')
newBCneg1 = Process(newBCneg)
# 处理iniBC
iniBC = np.load('initialUserBillChainP0.8.npy')
iniBC1 = Process(iniBC)
print('done')

## 将array还原成dataframe

In [ ]:
np.set_printoptions(suppress=True)
def arrayToDF(arr):
    res = []
    for lst in arr:
        res = res + list(lst)
    res_df = pd.DataFrame(res)
    res_df.columns = ['银行id','用户id','账单时间戳','上期账单金额','上期还款金额','信用卡额度','本期账单余额','还款状态']
    return res_df

In [ ]:
newBCpos_df = arrayToDF(newBCpos1)
newBCpos_df.to_csv('newBill_pos.csv', index = 0)

newBCneg_df = arrayToDF(newBCneg1)
newBCneg_df.to_csv('newBill_neg.csv', index = 0)

## 由于过摸较大要分开写，运行才会比较快，比如下面的可替换代码
iniBC_df = arrayToDF(iniBC1)
iniBC_df.to_csv('Bill.csv', index = 0)


In [ ]:
# 可替换的代码
iniBC1_1 = iniBC1[0:30000]
iniBC_df1 = arrayToDF(iniBC1_1)


In [ ]:
iniBC1_2 = iniBC1[30000:60000]
iniBC_df2 = arrayToDF(iniBC1_2)
#诸如此类